<a href="https://colab.research.google.com/github/katyayani-jha/ML-LAB-CS12/blob/main/ML_Lab_III.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Question 1.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score

In [2]:
from google.colab import drive
drive.mount('/content/drive')

import os

folder_path = '/content/drive/MyDrive'

import pandas as pd

file_path = os.path.join(folder_path, 'USA_Housing.csv')
df = pd.read_csv(file_path)
print(df.head())

Mounted at /content/drive
   Avg. Area Income  Avg. Area House Age  Avg. Area Number of Rooms  \
0       79545.45857             5.682861                   7.009188   
1       79248.64245             6.002900                   6.730821   
2       61287.06718             5.865890                   8.512727   
3       63345.24005             7.188236                   5.586729   
4       59982.19723             5.040555                   7.839388   

   Avg. Area Number of Bedrooms  Area Population         Price  
0                          4.09      23086.80050  1.059034e+06  
1                          3.09      40173.07217  1.505891e+06  
2                          5.13      36882.15940  1.058988e+06  
3                          3.26      34310.24283  1.260617e+06  
4                          4.23      26354.10947  6.309435e+05  


In [4]:
X = df.drop('Price', axis=1).values  # Input features
y = df['Price'].values  # Output variable

In [5]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [6]:
kf = KFold(n_splits=5)

# Initializing variables to track the best beta and best R² score
best_beta = None
best_r2_score = -np.inf

In [7]:
for train_index, test_index in kf.split(X_scaled):
    # Dividing the data into training and testing sets for each fold
    X_train, X_test = X_scaled[train_index], X_scaled[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Calculating the beta (𝛽) matrix using the normal equation: beta = (X.T * X)^(-1) * X.T * y
    X_train_bias = np.c_[np.ones(X_train.shape[0]), X_train]  # Adding the bias term (intercept)
    beta = np.linalg.inv(X_train_bias.T @ X_train_bias) @ X_train_bias.T @ y_train

    # Predicting on the test set
    X_test_bias = np.c_[np.ones(X_test.shape[0]), X_test]  # Adding the bias term for the test set
    y_pred = X_test_bias @ beta

    r2 = r2_score(y_test, y_pred)
    print(f'R² score for this fold: {r2}')

    if r2 > best_r2_score:
        best_r2_score = r2
        best_beta = beta

R² score for this fold: 0.9175899480765107
R² score for this fold: 0.920301549640113
R² score for this fold: 0.9152429915320014
R² score for this fold: 0.9208503836977656
R² score for this fold: 0.9138111758717495


In [8]:
from sklearn.model_selection import train_test_split

X_train_final, X_test_final, y_train_final, y_test_final = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

# Adding the bias term (intercept) for the final training and testing sets
X_train_final_bias = np.c_[np.ones(X_train_final.shape[0]), X_train_final]
X_test_final_bias = np.c_[np.ones(X_test_final.shape[0]), X_test_final]

# Calculating predictions using the best beta
y_train_pred = X_train_final_bias @ best_beta
y_test_pred = X_test_final_bias @ best_beta

final_r2_score = r2_score(y_test_final, y_test_pred)
print(f'Final R² score on test data: {final_r2_score}')

Final R² score on test data: 0.9146868482147401


Question 2.

In [9]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler

In [10]:
from google.colab import drive
drive.mount('/content/drive')

import os

folder_path = '/content/drive/MyDrive'

import pandas as pd

file_path = os.path.join(folder_path, 'USA_Housing.csv')
df = pd.read_csv(file_path)
print(df.head())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
   Avg. Area Income  Avg. Area House Age  Avg. Area Number of Rooms  \
0       79545.45857             5.682861                   7.009188   
1       79248.64245             6.002900                   6.730821   
2       61287.06718             5.865890                   8.512727   
3       63345.24005             7.188236                   5.586729   
4       59982.19723             5.040555                   7.839388   

   Avg. Area Number of Bedrooms  Area Population         Price  
0                          4.09      23086.80050  1.059034e+06  
1                          3.09      40173.07217  1.505891e+06  
2                          5.13      36882.15940  1.058988e+06  
3                          3.26      34310.24283  1.260617e+06  
4                          4.23      26354.10947  6.309435e+05  


In [11]:
X = df.drop('Price', axis=1).values
y = df['Price'].values

In [12]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [13]:
# Spliting the data into training (56%), validation (14%), and test (30%) sets
X_train, X_temp, y_train, y_temp = train_test_split(X_scaled, y, test_size=0.44, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.68, random_state=42)

In [14]:
def gradient_descent(X, y, learning_rate, iterations):
    m, n = X.shape
    theta = np.zeros(n)  # Initializing the regression coefficients (weights)
    for _ in range(iterations):
        gradient = (1/m) * X.T.dot(X.dot(theta) - y)
        theta -= learning_rate * gradient
    return theta

In [15]:
# Adding a bias term (intercept) to each set
X_train_bias = np.c_[np.ones(X_train.shape[0]), X_train]
X_val_bias = np.c_[np.ones(X_val.shape[0]), X_val]
X_test_bias = np.c_[np.ones(X_test.shape[0]), X_test]

In [16]:
learning_rates = [0.001, 0.01, 0.1, 1]
iterations = 1000
best_r2_val = -np.inf
best_theta = None
results = []

for lr in learning_rates:
    print(f"Training with learning rate: {lr}")

    # Training the model using gradient descent
    theta = gradient_descent(X_train_bias, y_train, lr, iterations)

    y_val_pred = X_val_bias.dot(theta)
    y_test_pred = X_test_bias.dot(theta)
    r2_val = r2_score(y_val, y_val_pred)
    r2_test = r2_score(y_test, y_test_pred)

    print(f"Learning Rate: {lr}, R² on Validation set: {r2_val}, R² on Test set: {r2_test}")

    results.append({
        'learning_rate': lr,
        'theta': theta,
        'r2_val': r2_val,
        'r2_test': r2_test
    })

    # Updating the best theta based on validation R² score
    if r2_val > best_r2_val:
        best_r2_val = r2_val
        best_theta = theta

best_result = max(results, key=lambda x: x['r2_val'])
print(f"\nBest learning rate: {best_result['learning_rate']}")
print(f"Best theta: {best_result['theta']}")
print(f"R² on Validation set: {best_result['r2_val']}")
print(f"R² on Test set: {best_result['r2_test']}")

Training with learning rate: 0.001
Learning Rate: 0.001, R² on Validation set: -1.0335339555142435, R² on Test set: -0.9634048206675561
Training with learning rate: 0.01
Learning Rate: 0.01, R² on Validation set: 0.9202011879628708, R² on Test set: 0.9133274876797325
Training with learning rate: 0.1
Learning Rate: 0.1, R² on Validation set: 0.9202207766800662, R² on Test set: 0.9133419747998835
Training with learning rate: 1
Learning Rate: 1, R² on Validation set: 0.9202207766800662, R² on Test set: 0.9133419747998835

Best learning rate: 0.1
Best theta: [1232180.27200919  230645.88389435  165328.94019375  120045.00851908
    2945.02108903  151375.22971285]
R² on Validation set: 0.9202207766800662
R² on Test set: 0.9133419747998835


Question 3.

In [17]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA
from sklearn.metrics import r2_score

In [18]:
columns = ["symboling", "normalized_losses", "make", "fuel_type", "aspiration",
           "num_doors", "body_style", "drive_wheels", "engine_location", "wheel_base",
           "length", "width", "height", "curb_weight", "engine_type", "num_cylinders",
           "engine_size", "fuel_system", "bore", "stroke", "compression_ratio",
           "horsepower", "peak_rpm", "city_mpg", "highway_mpg", "price"]

url = "https://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data"
df = pd.read_csv(url, names=columns, na_values='?')

In [21]:
imputer_mean = SimpleImputer(strategy='mean')
imputer_mode = SimpleImputer(strategy='most_frequent')

# Imputing numeric columns with mean
numeric_cols = df.select_dtypes(include=[np.number]).columns
df[numeric_cols] = imputer_mean.fit_transform(df[numeric_cols])

# Imputing categorical columns with mode
categorical_cols = df.select_dtypes(exclude=[np.number]).columns
df[categorical_cols] = imputer_mode.fit_transform(df[categorical_cols])

df = df.dropna(subset=['price'])

In [22]:
# Converting words to figures for 'num_doors' and 'num_cylinders'
num_doors_map = {'two': 2, 'four': 4}
num_cylinders_map = {'two': 2, 'three': 3, 'four': 4, 'five': 5, 'six': 6, 'eight': 8, 'twelve': 12}

df['num_doors'] = df['num_doors'].map(num_doors_map)
df['num_cylinders'] = df['num_cylinders'].map(num_cylinders_map)

# Dummy encoding for 'body_style' and 'drive_wheels'
df = pd.get_dummies(df, columns=['body_style', 'drive_wheels'], drop_first=True)

# Label encoding for 'make', 'aspiration', 'engine_location', 'fuel_type'
label_enc_cols = ['make', 'aspiration', 'engine_location', 'fuel_type']
le = LabelEncoder()
for col in label_enc_cols:
    df[col] = le.fit_transform(df[col])

# Replacing 'fuel_system': 'pfi' -> 1, else 0
df['fuel_system'] = np.where(df['fuel_system'].str.contains('pfi'), 1, 0)

# Replacing 'engine_type': 'ohc' -> 1, else 0
df['engine_type'] = np.where(df['engine_type'].str.contains('ohc'), 1, 0)

In [23]:
X = df.drop('price', axis=1).values
y = df['price'].values

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

regressor = LinearRegression()
regressor.fit(X_train, y_train)

y_pred = regressor.predict(X_test)
r2_original = r2_score(y_test, y_pred)
print(f"R² score on original test data: {r2_original}")

R² score on original test data: 0.804442243576259


In [25]:
pca = PCA(n_components=0.95)  # Retaining 95% of variance
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

# Training linear regressor on reduced data
regressor_pca = LinearRegression()
regressor_pca.fit(X_train_pca, y_train)

y_pred_pca = regressor_pca.predict(X_test_pca)
r2_pca = r2_score(y_test, y_pred_pca)
print(f"R² score on PCA-reduced test data: {r2_pca}")

if r2_pca > r2_original:
    print("PCA improved the model performance.")
else:
    print("PCA did not improve the model performance.")

R² score on PCA-reduced test data: 0.7577769105521941
PCA did not improve the model performance.
